In [9]:
# Operações para Silver

from pyspark.sql import *
from delta import *

builder = SparkSession.builder.appName("topicos").config("spark.sql.extensions","io.delta.sql.DeltaSparkSessionExtension").config("spark.sql.catalog.spark_catalog","org.apache.spark.sql.delta.catalog.DeltaCatalog")
spark = configure_spark_with_delta_pip(builder).getOrCreate()

INPUT_PATH = "/home/felipe/code/topicos_dados/dados/"
BRONZE_PATH = "/home/felipe/code/deltalake/lake/bronze/"
SILVER_PATH = "/home/felipe/code/deltalake/lake/silver/"

print(spark.version)


3.5.0


In [18]:
from pyspark.sql import *
from delta import *

builder = SparkSession.builder.appName("topicos").config("spark.sql.extensions","io.delta.sql.DeltaSparkSessionExtension").config("spark.sql.catalog.spark_catalog","org.apache.spark.sql.delta.catalog.DeltaCatalog")
spark = configure_spark_with_delta_pip(builder).getOrCreate()


brbus_table = DeltaTable.create(spark) \
    .tableName("silver_brbus") \
    .addColumn("latitude", dataType="FLOAT", nullable=False) \
    .addColumn("longitude", dataType="FLOAT") \
    .addColumn("bus_id", dataType="STRING") \
    .addColumn("city", dataType="STRING") \
    .addColumn("bus_code", dataType="STRING") \
    .addColumn("bus_speed", dataType="FLOAT") \
    .addColumn("bus_direction", dataType="INT") \
    .addColumn("queried_at", dataType="STRING") \
    .addColumn("updated_at", dataType="STRING") \
    .location("/home/felipe/code/topicos_dados/lake/silver/silver_brbus") \
    .execute()



In [3]:
from deltalake import DeltaTable

teste = spark.read.format("delta").load("/home/felipe/code/topicos_dados/lake/silver/silver_cwb/")
teste.show()

24/03/12 11:24:00 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----------+----------+--------------+--------+----------+---------------+------+----------+--------------------+---------+-------------------+-------------+
|  latitude| longitude|  tipo_veiculo|bus_code|  situacao|     situacao_2|tabela|is_adapted|          queried_at|   bus_id|         updated_at|bus_direction|
+----------+----------+--------------+--------+----------+---------------+------+----------+--------------------+---------+-------------------+-------------+
|-25.439138|-49.272285|MICRO ESPECIAL|     762|  ATRASADO|REALIZANDO ROTA|   2-2|         1|2024-01-30T23:19:21Z|CWB_JI859|2024-01-3023:18:00Z|            1|
|-25.442966|-49.245985|MICRO ESPECIAL|     463|NO HORÁRIO|REALIZANDO ROTA|     2|         1|2024-01-30T23:19:21Z|CWB_DN603|2024-01-3023:19:00Z|            2|
|-25.457181|-49.222675|MICRO ESPECIAL|     463|NO HORÁRIO|REALIZANDO ROTA|     4|         1|2024-01-30T23:19:21Z|CWB_DN608|2024-01-3023:19:00Z|            1|
|-25.441758|-49.347495|    ARTICULADO|     040|  ATR

In [19]:
from deltalake import DeltaTable

table_brbus = spark.read.format("delta").load("/home/felipe/code/topicos_dados/lake/silver/silver_brbus/")
table_brbus.show()

+--------+---------+------+----+--------+---------+-------------+----------+----------+
|latitude|longitude|bus_id|city|bus_code|bus_speed|bus_direction|queried_at|updated_at|
+--------+---------+------+----+--------+---------+-------------+----------+----------+
+--------+---------+------+----+--------+---------+-------------+----------+----------+



In [22]:
# Adding city values to each DF

from pyspark.sql.functions import lit
# SP

sp_silver = spark.read.format("delta").load("/home/felipe/code/topicos_dados/lake/silver/silver_sp/")
sp_silver = sp_silver.withColumn("city",lit("Sao Paulo"))
sp_selected = sp_silver.select(["longitude", "latitude","bus_code","queried_at","updated_at","bus_id"])


# RJ
rj_silver = spark.read.format("delta").load("/home/felipe/code/topicos_dados/lake/silver/silver_rj/")
rj_silver = rj_silver.withColumn("city",lit("Rio de Janeiro"))
rj_selected = rj_silver.select(["longitude", "latitude","bus_speed","bus_code","queried_at","updated_at","bus_id"])

# CWB
cwb_silver = spark.read.format("delta").load("/home/felipe/code/topicos_dados/lake/silver/silver_cwb/")
cwb_silver = cwb_silver.withColumn("city",lit("Curitiba"))
cwb_selected = cwb_silver.select(["longitude", "latitude","bus_code","queried_at","updated_at","bus_id","bus_direction"])

#BSB

bsb_silver = spark.read.format("delta").load("/home/felipe/code/topicos_dados/lake/silver/silver_bsb/")
bsb_silver = bsb_silver.withColumn("city",lit("Brasilia"))

bsb_selected = bsb_silver.select(["longitude", "latitude","bus_speed","bus_code","queried_at","updated_at","bus_id","bus_direction"])

bsb_selected.show()
# Merging into BRBus Table



+---------+---------+---------+--------+--------------------+--------------------+----------+-------------+
|longitude| latitude|bus_speed|bus_code|          queried_at|          updated_at|    bus_id|bus_direction|
+---------+---------+---------+--------+--------------------+--------------------+----------+-------------+
|-47.90188|-15.78581|    16.67|   0.882|2024-01-30T23:24:31Z|2024-01-301706665...|BSB_336921|            2|
|-47.95254| -15.8094|    18.61|   0.813|2024-01-30T23:24:31Z|2024-01-301706667...|BSB_336408|            2|
|-47.89276| -15.8111|     6.94|   0.373|2024-01-30T23:24:31Z|2024-01-301706667...|BSB_340103|            1|
| -48.1512| -15.8845|      0.0|   373.2|2024-01-30T23:24:31Z|2024-01-301706667...|BSB_335991|            1|
|-47.95401|-15.84706|    16.94|   0.821|2024-01-30T23:24:31Z|2024-01-301706667...|BSB_335487|            1|
|-48.15105|-15.88424|     0.56|   0.373|2024-01-30T23:24:31Z|2024-01-301706667...|BSB_340065|            1|
|-48.04572|  -15.853|      0